In [173]:
import json

import numpy as np
import pandas as pd
import pymongo
import scipy
from bson import ObjectId, json_util
from pandas.io.json import json_normalize
from pymongo import MongoClient as Connection
from scipy import sparse, spatial
from scipy.spatial.distance import cdist, pdist, squareform
from sklearn.metrics.pairwise import cosine_similarity

# scientific notation disabled form smaller numbers
pd.options.display.float_format = '{:.5f}'.format

# alles resultate anzeigen und nicht nur das letzte
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [174]:
# display multiple outputs in one row
import pandas as pd
import numpy as np
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

In [175]:
connection = Connection()
db = connection.recipe_db
input_data = db.recipes_test_copy

data = json.loads(json_util.dumps(input_data.find()))

In [176]:
ing = pd.DataFrame(json_normalize(data, record_path='ingredients',
                             meta='id', record_prefix='ingredients_', errors='ignore'))


nutritions = pd.DataFrame(json_normalize(data, record_path='nutritions',
                            meta=['id', 'prep_time', 'rating', 'rating_count', 'ready_in_time', 'review_count']))


In [177]:
#ingredients.loc[ingredients['id'] == 9380]

In [178]:
#ingredients.loc[ingredients['ingredients_id'] == 2972]

In [179]:
#  ------  erstellung und data cleansing - Ingredients

# schmeiss alle zutaten raus, die weniger als 5 mal verwendet werden
# setze multiinde auf 'id' und 'ingredients_id'
ingredients = ing.set_index(['id', 'ingredients_id'])

# filtere alle Zutaten samit ihrer rezepte id, die weniger gleich 5 mal vorkommen
ingredients_eqles_5_ing = ingredients.groupby(
    'ingredients_id').filter(lambda x: len(x) <= 5)

# droppe alle rezepte, die eine Zutate besitzen, die weniger gleich 5 Mal vorkommen
ingredients_filt = ingredients.drop(ingredients_eqles_5_ing.index.get_level_values('id').values, level=0)

# drop alls rows with ingredients_id == 0
ingredients_eqal_zero = ingredients_filt[ingredients_filt.index.get_level_values('ingredients_id') == 0]
ingredients_filt = ingredients_filt.drop(ingredients_eqal_zero.index.get_level_values('id').values, level=0)


In [180]:
#  ------ Erstellung und cleansing des Nutrition Dataframes

# erstelle neue liste auf basis der bereits gefilterten rezepte aus ingredients_filt
id_overlap_mask = nutritions['id'].isin(ingredients_filt.index.get_level_values('id').values)

# erstelle datenframe auf basis der overlapliste
nutritions_filt = nutritions.loc[id_overlap_mask]

nutrition_db = nutritions_filt.pivot_table(
    index=['id'],
    columns=['name'],
    values=['amount'],
).reset_index()

nutrition_db.set_index('id', inplace=True)

# remove multiindex 'amount'
nutrition_db.columns = nutrition_db.columns.droplevel(0)

# entferne alle NA
nutrition_db = nutrition_db.dropna()

# gleiche nochmals die ids der beiden dataframe nutrition und ingredients ab, da der nutrition dataframe noch NA Werte hatt
id_overlap_mask = ingredients_filt.index.get_level_values('id').isin(nutrition_db.index)
ingredients_db = ingredients_filt[id_overlap_mask]

# abgleich ob anzahl der indizes von nutrition und zutaten dataframe gleich sind
nutrition_db.index.nunique()
ingredients_db.index.get_level_values('id').nunique()
nutrition_db

4061

4061

name,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,Niacin Equivalents,Potassium,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
id,,,,,,,,,,,,,,,,,,,,
6900,117.79860,105.27020,21.33330,17.98829,1.62667,0.56557,2.37037,39.16360,1.14346,7.30480,1.82102,54.07343,2.84282,0.59495,385.75980,1.72141,0.17169,116.75390,0.01709,0.04505
7198,244.16520,731.50920,325.65230,73.29574,88.69389,4.64147,36.18359,249.14210,5.76566,52.89296,13.51325,509.34610,27.58195,11.50691,1660.20300,2.83424,0.89150,699.95980,0.36319,67.32533
8493,415.52450,454.28300,177.75310,23.83120,203.86850,1.99500,19.75034,63.05025,3.11258,54.35775,23.43780,509.41500,44.07647,9.96063,1107.91100,3.45304,0.17406,578.21420,0.64563,1.91250
8494,58.08641,834.44480,514.68470,4.76954,283.75000,0.63600,57.18719,33.44816,3.72336,84.44925,39.18463,807.96980,71.10712,16.35430,567.97140,2.08741,0.02921,619.52510,1.37244,20.17546
8495,379.01410,418.75340,174.23740,12.62089,123.58900,0.75239,19.35971,25.41156,1.89411,60.48306,24.62857,457.03550,46.06678,10.03598,931.58340,1.42275,0.29605,406.88370,0.82640,11.72893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258878,9.30933,189.09670,106.42770,0.12118,70.99000,0.01650,11.82530,5.34667,1.02767,16.89067,8.44069,171.30030,19.15713,3.30546,90.00200,0.04050,0.05192,125.95000,0.24153,0.03000
259870,116.87750,533.78500,254.83060,65.83649,0.00000,12.46288,28.31451,179.44000,4.86495,140.72750,7.50368,2441.57700,11.27299,4.00073,141.41000,16.18104,0.55845,2659.15000,1.63708,135.49670
260193,37.24250,300.08690,43.28884,13.86486,118.04670,1.03600,4.80987,7.15000,1.63545,46.19250,21.66052,476.54750,45.85212,1.41095,1545.54100,4.58523,0.08974,217.95000,0.61491,1.23025


In [181]:
ingredients_db.loc[ingredients_db['id'] == 20881]

KeyError: 'id'

In [0]:
id_list = [215014, 8669, 16700, 16354, 12720, 8652, 8887, 51283, 45954, 213742, 14595,
           219164, 16348, 143082, 8665, 11758, 223042, 236609, 8778, 65896, 24264, 11679, 141678, 9023]

#ingredients_db.reset_index(inplace=True)

recipe_db = pd.get_dummies(ingredients_db['ingredients_id']).groupby(
    ingredients_db['id']).apply(max)



In [198]:
new_ingredients = ingredients_db
new_ingredients = new_ingredients.groupby("id")["ingredients_grams"].sum().reset_index()

In [199]:
new_ingredients.sort_values(by='ingredients_grams')

,id,ingredients_grams
1214,20881,21.22222
2493,79255,73.29343
2831,106030,79.13929
1788,34531,91.28767
410,12063,107.10890
...,...,...
2137,55946,4711.39585
3734,229874,4731.00000
2063,51849,4780.40889
3838,232865,5779.31317


In [200]:
new_ingredients['quotient'] = new_ingredients['ingredients_grams']/500

In [0]:
new_ingredients.set

In [201]:
new_ingredients.describe()

,id,ingredients_grams,quotient
count,4061.00000,4061.00000,4061.00000
mean,86301.50874,1158.85360,2.31771
std,82174.58263,536.42903,1.07286
min,6900.00000,21.22222,0.04244
25%,18054.00000,790.07346,1.58015
50%,50054.00000,1071.33340,2.14267
75%,152211.00000,1425.14042,2.85028
max,263813.00000,7908.48667,15.81697


In [0]:
new_ingredients.hist(column='ingredients_grams', bins=200)

In [0]:
# rechne nährwerte auf 800 kcal pro rezept um
def nutrition_to_500(x):
    a = x / x
    b = a * 800
    return b

In [0]:
nutrition_db_800 = nutrition_db.apply(nutrition_to_800, axis = 1)
#user_nutrition = nutrition_db_800[nutrition_db_800.index.isin(id_list)]

user_recipes = recipe_db[recipe_db.index.isin(id_list)]

recommenden_nut = pd.DataFrame(data=None, columns=nutrition_db_800.columns, index=None)


In [220]:
new_nutrition = nutrition_db.copy()

In [206]:
new_ingredients.set_index('id', inplace=True)

In [207]:
new_ingredients

,ingredients_grams,quotient
id,,
6900,184.35112,0.36870
7198,1238.50889,2.47702
8493,1280.50000,2.56100
8494,1671.56307,3.34313
8495,817.26250,1.63452
...,...,...
258878,456.93333,0.91387
259870,3484.50000,6.96900
260193,1464.50000,2.92900


In [221]:
new_nutrition

name,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,Niacin Equivalents,Potassium,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
id,,,,,,,,,,,,,,,,,,,,
6900,117.79860,105.27020,21.33330,17.98829,1.62667,0.56557,2.37037,39.16360,1.14346,7.30480,1.82102,54.07343,2.84282,0.59495,385.75980,1.72141,0.17169,116.75390,0.01709,0.04505
7198,244.16520,731.50920,325.65230,73.29574,88.69389,4.64147,36.18359,249.14210,5.76566,52.89296,13.51325,509.34610,27.58195,11.50691,1660.20300,2.83424,0.89150,699.95980,0.36319,67.32533
8493,415.52450,454.28300,177.75310,23.83120,203.86850,1.99500,19.75034,63.05025,3.11258,54.35775,23.43780,509.41500,44.07647,9.96063,1107.91100,3.45304,0.17406,578.21420,0.64563,1.91250
8494,58.08641,834.44480,514.68470,4.76954,283.75000,0.63600,57.18719,33.44816,3.72336,84.44925,39.18463,807.96980,71.10712,16.35430,567.97140,2.08741,0.02921,619.52510,1.37244,20.17546
8495,379.01410,418.75340,174.23740,12.62089,123.58900,0.75239,19.35971,25.41156,1.89411,60.48306,24.62857,457.03550,46.06678,10.03598,931.58340,1.42275,0.29605,406.88370,0.82640,11.72893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258878,9.30933,189.09670,106.42770,0.12118,70.99000,0.01650,11.82530,5.34667,1.02767,16.89067,8.44069,171.30030,19.15713,3.30546,90.00200,0.04050,0.05192,125.95000,0.24153,0.03000
259870,116.87750,533.78500,254.83060,65.83649,0.00000,12.46288,28.31451,179.44000,4.86495,140.72750,7.50368,2441.57700,11.27299,4.00073,141.41000,16.18104,0.55845,2659.15000,1.63708,135.49670
260193,37.24250,300.08690,43.28884,13.86486,118.04670,1.03600,4.80987,7.15000,1.63545,46.19250,21.66052,476.54750,45.85212,1.41095,1545.54100,4.58523,0.08974,217.95000,0.61491,1.23025


In [247]:
new_nutrition1 = new_nutrition.merge(new_ingredients, how='left', on='id')


In [248]:
new_nutrition1

,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,...,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C,ingredients_grams,quotient
id,,,,,,,,,,,,,,,,,,,,,
6900,117.79860,105.27020,21.33330,17.98829,1.62667,0.56557,2.37037,39.16360,1.14346,7.30480,...,2.84282,0.59495,385.75980,1.72141,0.17169,116.75390,0.01709,0.04505,184.35112,0.36870
7198,244.16520,731.50920,325.65230,73.29574,88.69389,4.64147,36.18359,249.14210,5.76566,52.89296,...,27.58195,11.50691,1660.20300,2.83424,0.89150,699.95980,0.36319,67.32533,1238.50889,2.47702
8493,415.52450,454.28300,177.75310,23.83120,203.86850,1.99500,19.75034,63.05025,3.11258,54.35775,...,44.07647,9.96063,1107.91100,3.45304,0.17406,578.21420,0.64563,1.91250,1280.50000,2.56100
8494,58.08641,834.44480,514.68470,4.76954,283.75000,0.63600,57.18719,33.44816,3.72336,84.44925,...,71.10712,16.35430,567.97140,2.08741,0.02921,619.52510,1.37244,20.17546,1671.56307,3.34313
8495,379.01410,418.75340,174.23740,12.62089,123.58900,0.75239,19.35971,25.41156,1.89411,60.48306,...,46.06678,10.03598,931.58340,1.42275,0.29605,406.88370,0.82640,11.72893,817.26250,1.63452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258878,9.30933,189.09670,106.42770,0.12118,70.99000,0.01650,11.82530,5.34667,1.02767,16.89067,...,19.15713,3.30546,90.00200,0.04050,0.05192,125.95000,0.24153,0.03000,456.93333,0.91387
259870,116.87750,533.78500,254.83060,65.83649,0.00000,12.46288,28.31451,179.44000,4.86495,140.72750,...,11.27299,4.00073,141.41000,16.18104,0.55845,2659.15000,1.63708,135.49670,3484.50000,6.96900
260193,37.24250,300.08690,43.28884,13.86486,118.04670,1.03600,4.80987,7.15000,1.63545,46.19250,...,45.85212,1.41095,1545.54100,4.58523,0.08974,217.95000,0.61491,1.23025,1464.50000,2.92900


In [226]:
#new_nutrition1 = new_nutrition1.drop('ingredients_grams',1)

In [227]:
new_nutrition1

,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,...,Potassium,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C,quotient
id,,,,,,,,,,,,,,,,,,,,,
6900,117.79860,105.27020,21.33330,17.98829,1.62667,0.56557,2.37037,39.16360,1.14346,7.30480,...,54.07343,2.84282,0.59495,385.75980,1.72141,0.17169,116.75390,0.01709,0.04505,0.36870
7198,244.16520,731.50920,325.65230,73.29574,88.69389,4.64147,36.18359,249.14210,5.76566,52.89296,...,509.34610,27.58195,11.50691,1660.20300,2.83424,0.89150,699.95980,0.36319,67.32533,2.47702
8493,415.52450,454.28300,177.75310,23.83120,203.86850,1.99500,19.75034,63.05025,3.11258,54.35775,...,509.41500,44.07647,9.96063,1107.91100,3.45304,0.17406,578.21420,0.64563,1.91250,2.56100
8494,58.08641,834.44480,514.68470,4.76954,283.75000,0.63600,57.18719,33.44816,3.72336,84.44925,...,807.96980,71.10712,16.35430,567.97140,2.08741,0.02921,619.52510,1.37244,20.17546,3.34313
8495,379.01410,418.75340,174.23740,12.62089,123.58900,0.75239,19.35971,25.41156,1.89411,60.48306,...,457.03550,46.06678,10.03598,931.58340,1.42275,0.29605,406.88370,0.82640,11.72893,1.63452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258878,9.30933,189.09670,106.42770,0.12118,70.99000,0.01650,11.82530,5.34667,1.02767,16.89067,...,171.30030,19.15713,3.30546,90.00200,0.04050,0.05192,125.95000,0.24153,0.03000,0.91387
259870,116.87750,533.78500,254.83060,65.83649,0.00000,12.46288,28.31451,179.44000,4.86495,140.72750,...,2441.57700,11.27299,4.00073,141.41000,16.18104,0.55845,2659.15000,1.63708,135.49670,6.96900
260193,37.24250,300.08690,43.28884,13.86486,118.04670,1.03600,4.80987,7.15000,1.63545,46.19250,...,476.54750,45.85212,1.41095,1545.54100,4.58523,0.08974,217.95000,0.61491,1.23025,2.92900


In [249]:
new_nutrition2 = new_nutrition1.copy()

In [244]:
new_nutrition2.iloc[:, 0:19+1]

,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,Niacin Equivalents,Potassium,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C
id,,,,,,,,,,,,,,,,,,,,
6900,117.79860,105.27020,21.33330,17.98829,1.62667,0.56557,2.37037,39.16360,1.14346,7.30480,1.82102,54.07343,2.84282,0.59495,385.75980,1.72141,0.17169,116.75390,0.01709,0.04505
7198,244.16520,731.50920,325.65230,73.29574,88.69389,4.64147,36.18359,249.14210,5.76566,52.89296,13.51325,509.34610,27.58195,11.50691,1660.20300,2.83424,0.89150,699.95980,0.36319,67.32533
8493,415.52450,454.28300,177.75310,23.83120,203.86850,1.99500,19.75034,63.05025,3.11258,54.35775,23.43780,509.41500,44.07647,9.96063,1107.91100,3.45304,0.17406,578.21420,0.64563,1.91250
8494,58.08641,834.44480,514.68470,4.76954,283.75000,0.63600,57.18719,33.44816,3.72336,84.44925,39.18463,807.96980,71.10712,16.35430,567.97140,2.08741,0.02921,619.52510,1.37244,20.17546
8495,379.01410,418.75340,174.23740,12.62089,123.58900,0.75239,19.35971,25.41156,1.89411,60.48306,24.62857,457.03550,46.06678,10.03598,931.58340,1.42275,0.29605,406.88370,0.82640,11.72893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258878,9.30933,189.09670,106.42770,0.12118,70.99000,0.01650,11.82530,5.34667,1.02767,16.89067,8.44069,171.30030,19.15713,3.30546,90.00200,0.04050,0.05192,125.95000,0.24153,0.03000
259870,116.87750,533.78500,254.83060,65.83649,0.00000,12.46288,28.31451,179.44000,4.86495,140.72750,7.50368,2441.57700,11.27299,4.00073,141.41000,16.18104,0.55845,2659.15000,1.63708,135.49670
260193,37.24250,300.08690,43.28884,13.86486,118.04670,1.03600,4.80987,7.15000,1.63545,46.19250,21.66052,476.54750,45.85212,1.41095,1545.54100,4.58523,0.08974,217.95000,0.61491,1.23025


In [233]:
new_nutrition2.iloc[:, 1:19+1] *= new_nutrition2['quotient']

In [250]:
new_nutrition2.update(new_nutrition2.iloc[:, 0:19+1].mul(new_nutrition2.quotient, 0))

In [251]:
new_nutrition2

,Calcium,Calories,Calories from Fat,Carbohydrates,Cholesterol,Dietary Fiber,Fat,Folate,Iron,Magnesium,...,Protein,Saturated Fat,Sodium,Sugars,Thiamin,Vitamin A - IU,Vitamin B6,Vitamin C,ingredients_grams,quotient
id,,,,,,,,,,,,,,,,,,,,,
6900,43.43261,38.81336,7.86564,6.63232,0.59976,0.20853,0.87396,14.43971,0.42160,2.69330,...,1.04816,0.21936,142.23050,0.63469,0.06330,43.04742,0.00630,0.01661,184.35112,0.36870
7198,604.80154,1811.96129,806.64654,181.55485,219.69634,11.49699,89.62740,617.12941,14.28164,131.01680,...,68.32098,28.50282,4112.35235,7.02047,2.20826,1733.81287,0.89963,166.76604,1238.50889,2.47702
8493,1064.15824,1163.41876,455.22569,61.03170,522.10723,5.10920,50.58062,161.47169,7.97130,139.21020,...,112.87984,25.50918,2837.36007,8.84324,0.44577,1480.80657,1.65346,4.89791,1280.50000,2.56100
8494,194.19020,2789.65422,1720.65587,15.94519,948.61204,2.12624,191.18399,111.82142,12.44765,282.32449,...,237.72007,54.67449,1898.80003,6.97847,0.09765,2071.15055,4.58824,67.44911,1671.56307,3.34313
8495,619.50802,684.46290,284.79539,20.62916,202.00931,1.22980,31.64393,41.53583,3.09597,98.86107,...,75.29730,16.40406,1522.69636,2.32552,0.48390,665.06158,1.35078,19.17123,817.26250,1.63452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258878,8.50749,172.80917,97.26073,0.11075,64.87539,0.01508,10.80675,4.88614,0.93915,15.43582,...,17.50706,3.02075,82.24983,0.03701,0.04745,115.10151,0.22073,0.02742,456.93333,0.91387
259870,814.51930,3719.94767,1775.91445,458.81450,0.00000,86.85381,197.32382,1250.51736,33.90384,980.72995,...,78.56147,27.88105,985.48629,112.76567,3.89186,18531.61635,11.40879,944.27650,3484.50000,6.96900
260193,109.08328,878.95453,126.79301,40.61017,345.75878,3.03444,14.08811,20.94235,4.79023,135.29783,...,134.30086,4.13268,4526.88959,13.43015,0.26285,638.37555,1.80108,3.60340,1464.50000,2.92900
